In [ ]:
# Detecting face
import numpy as np
import cv2

detector= cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(gray, 1.5, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

    cv2.imshow('frame',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
### Enhancing code, no nid detect face only open window
# Getting dataset

import numpy as np
import cv2,sqlite3

cam = cv2.VideoCapture(0)
detector=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#define function
def insertOrUpdate(Id,Name,Age,Description):
    conn=sqlite3.connect("FaceDB.db")
    cmd="SELECT * FROM PERSON WHERE ID="+str(Id)
    cursor=conn.execute(cmd)
    isRecordExist=0
    for row in cursor:
        isRecordExist=1
    if(isRecordExist==1):
        cmd="UPDATE PERSON SET Name='"+str(Name)+" ', Age=' "+str(Age)+" ',Description='"+str(Description)+" ' WHERE ID="+str(Id)
    else:
        cmd="INSERT INTO PERSON(ID,Age,Name,Description) Values("+str(Id)+","+str(Age)+",'"+str(Name)+" ','"+str(Description)+" ' )"
    conn.execute(cmd)
    conn.commit()
    conn.close()
    

Id=input('enter your id')
Name=input('enter your name')
Age=input('enter your age')
Description=input('enter your description')
insertOrUpdate(Id,Name,Age,Description)

sampleNum=0
if cam.isOpened():
    while(cam.isOpened()):  # check !
        # capture frame-by-frame
        ret, img = cam.read()
        cv2.imshow('frame', img)

        if ret: # check ! (some webcam's need a "warmup")
            # our operation on frame come here
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = detector.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

                #incrementing sample number 
                sampleNum=sampleNum+1
                #saving the captured face in the dataset folder
                cv2.imwrite("dataSet/User."+Id +'.'+ str(sampleNum) + ".jpg", gray[y:y+h,x:x+w])
                # Display the resulting frame
                cv2.imshow('frame', img)

        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
            # break if the sample number is morethan 20
        elif sampleNum>=40:
            break
    # When everything is done release the capture
cam.release()
cv2.destroyAllWindows()
print("Data set updated")

enter your id1


In [4]:
#Training new face
import cv2,os
import numpy as np
from PIL import Image

#recognizer = cv2.face.FisherFaceRecognizer_create()
#recognizer = cv2.face.EigenFaceRecognizer_create()
recognizer = cv2.face.LBPHFaceRecognizer_create()
#help(cv2.face)
detector= cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

def getImagesAndLabels(path):
    #get the path of all the files in the folder
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)] 
    #create empth face list
    faceSamples=[]
    #create empty ID list
    Ids=[]
    #now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:

        # Updates in Code
        # ignore if the file does not have jpg extension :
        if(os.path.split(imagePath)[-1].split(".")[-1]!='jpg'):
            continue

        #loading the image and converting it to gray scale
        pilImage=Image.open(imagePath).convert('L')        
        #Now we are converting the PIL image into numpy array
        imageNp=np.array(pilImage,'uint8')
        #getting the Id from the image
        Id=int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces=detector.detectMultiScale(imageNp)
        #If a face is there then append that in the list as well as Id of it
        for (x,y,w,h) in faces:
            faceSamples.append(imageNp[y:y+h,x:x+w])
            Ids.append(Id)
    return faceSamples,Ids


faces,Ids = getImagesAndLabels('dataSet')
recognizer.train(faces, np.array(Ids))
recognizer.write('trainner/trainner.yml')
print('Face Trainned')

Face Trainned


In [5]:
#Detecting the face
import cv2,sqlite3
import numpy as np

#recognizer = cv2.face.createFisherFaceRecognizer()
#recognizer = cv2.face.createEigenFaceRecognizer()
recognizer = cv2.face.LBPHFaceRecognizer_create()

recognizer.read('trainner/trainner.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

#def function
def getProfile(Id):
    conn=sqlite3.connect("FaceDB.db")
    cmd="SELECT * FROM PERSON WHERE ID="+str(Id)
    cursor = conn.execute(cmd)
    profile=None
    for row in cursor:
        profile=row
    conn.close()
    return profile


cap = cv2.VideoCapture(0)
fontface = cv2.FONT_HERSHEY_SIMPLEX
fontscale = 1
fontcolor = (54,67,244)

if(cap.isOpened()):    
    while(cap.isOpened()):
        ret, img = cap.read()
        #cv2.imshow('frame',img)
        if ret:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(gray, 1.1, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                Id, conf = recognizer.predict(gray[y:y+h,x:x+w])
                profile = getProfile(Id)
                if(profile!=None):
                    cv2.putText(img, str(profile[1]), (x,y+h+30), fontface, fontscale, fontcolor)
                    cv2.putText(img, str(profile[2]), (x,y+h+60), fontface, fontscale, fontcolor)
                    cv2.putText(img, str(profile[3]), (x,y+h+90), fontface, fontscale, fontcolor)                       
                    cv2.putText(img, str(conf), (x,y+h+120), fontface, fontscale, fontcolor) 
            cv2.imshow('frame',img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

print('ok')

ok


In [ ]:
#Delete 1 row database
import sqlite3

def clearRowDB(Id):
    conn=sqlite3.connect("FaceDB.db")
    cmd="SELECT * FROM PERSON WHERE ID="+str(Id)
    cursor=conn.execute(cmd)
    isRecordExist=0
    for row in cursor:
        isRecordExist=1
    if(isRecordExist==1):
        cmd="DELETE FROM PERSON WHERE ID="+str(Id)
        print("Row deleted")
    else:
        print("No record found")
    conn.execute(cmd)
    conn.commit()
    conn.close()
    
Id=input("Enter deleted row id : ")
clearRowDB(Id)


In [2]:
#Clear databse
import sqlite3

def clearDB():
    conn=sqlite3.connect("FaceDB.db")
    cmd="DELETE FROM PERSON"
    conn.execute(cmd)
    conn.commit()
    conn.close()
    

clearDB()
print("Database clear")

Database clear
